In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm

# hyperparameters
batch_size = 64
block_size = 256
n_embed = 384
n_head = 6
n_layer = 6
epochs = 5000
epoch_step = 500
dropout = 0.2
learning_rate = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"

## Prepare Data

In [ ]:
with open('data/TinyShakespeare/text.txt', 'r', encoding='utf-8') as file:
    text = file.read()

Build Vocabulary

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
''.join(chars)

Character Tokenization Functions

In [ ]:
stoi = { ch: i for i, ch in enumerate(chars) }
itos = { i: ch for i, ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print("test ->", encode("test"), "->", decode(encode("test")))

Encode Data

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long, device=device)
data.shape, data[:100]

Split data into train and test

In [ ]:
split = int(0.9 * len(data))
train_data = data[:split]
val_data = data[split:]
len(train_data), len(val_data)

In [ ]:
train_data[:block_size+1]

In [ ]:
for s in range(block_size):
    context = train_data[:s+1].cpu().numpy().tolist()
    target = train_data[s+1].item()
    print(context, "->", target)

In [ ]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]).to(device)
    
    return x, y

xb, yb = get_batch('train')
print('context:\n', xb, '\ntargets:\n', yb)

## Build Model

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k, q, v = self.key(x), self.query(x), self.value(x)
        wei = q @ k.transpose(-2, -1) * C ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        out = wei @ v
        return out
    
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layer)])
        self.ln = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        token_embed = self.token_embedding_table(idx)
        position_embed = self.position_embedding_table(torch.arange(T, device=device))
        x = token_embed + position_embed
        x = self.blocks(x)
        x = self.ln(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)

            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim=-1)

            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx
    
model = BigramLanguageModel(vocab_size).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

Test prediction starting with a single zero

In [ ]:
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(idx, max_new_tokens=1000)[0].tolist()))

## Train Model

In [ ]:
xb, yb = get_batch('train')
logits, starting_loss = model(xb, yb)

for epoch in tqdm(range(epochs)):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch % epoch_step == 0):
        print(f'Loss at epoch {epoch}: loss')

print("Starting Loss:", starting_loss)
print("Ending Loss:", loss)


In [ ]:
# Examples of averaging past context
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)

In [ ]:
## Inefficient way (using for loops)
xbow = torch.zeros((B, T, C))

for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

x[0], xbow[0]

In [ ]:
## Efficient way (using math)
wei = torch.tril(torch.ones(T, T))
wei = wei / torch.sum(wei, 1, keepdim=True)
xbow = wei @ x

x[0], xbow[0]

In [ ]:
## Another way (allows self-attention)
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow = wei @ x

x[0], xbow[0]

In [ ]:
## Test for performing self-attention on single head
B, T, C = 4, 8, 32
torch.manual_seed(100)
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k, q, v = key(x), query(x), value(x)
wei = q @ k.transpose(-2, -1) * head_size ** -.5

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ v

v[0,:, 0], out[0,:, 0]

In [ ]:
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(idx, max_new_tokens=1000)[0].tolist()))